In [2]:
# only needed for colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
# os.chdir('/content/drive/MyDrive/TaylorExpansionML')
os.chdir("..")

In [4]:
import numpy as np
import pickle
import sympy as sp
from sympy import sympify, srepr
import os
from icecream import ic
import random

from source.data_preparation import sympy_to_prefix, prefix_to_sympy, vectorize_ds, vectorize_sentence, pad_right
import functools
from sklearn.model_selection import train_test_split

In [5]:
from tqdm.notebook import trange, tqdm

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras import layers

2022-06-01 12:06:34.186988: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-01 12:06:34.189602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 12:06:34.189612: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
from gensim.models import Word2Vec
from icecream import ic

In [8]:
load_data_from_scratch = False
export_data = False
load_full_data = True
check_prefix_recovery = False

In [9]:
if load_data_from_scratch:
    data_file = "data.nosync/data.txt"
    taylor_file = "data.nosync/data_taylor.txt"
    coeffs_file = "data.nosync/data_coeffs.txt"

    start = ["[start]"]
    end = ["[end]"]

    with open(data_file) as f:
        X = f.read().split("\n")
        X = np.array(X)
        X = X[:-1]   # somehow last entry is empty
        X = [sympify(xi) for xi in X]

    with open(taylor_file) as f:
        y_taylor = f.read().split("\n")
        y_taylor = np.array(y_taylor)
        y_taylor = y_taylor[:-1]
        y_taylor = [sympify(yi) for yi in y_taylor]

    with open(coeffs_file) as f:
        y_coeffs = f.read().split("\n")
        y_coeffs = y_coeffs[:-1]
        for i, y in enumerate(y_coeffs):
            y = sympify(y)
            # add start end tokens and remove whitespaces
            y_coeffs[i] = y

In [10]:
if load_data_from_scratch:
    train_val_test = [0.8, 0.1, 0.1]
    assert np.sum(train_val_test) == 1
    assert len(X) == len(y_taylor)
    assert len(X) == len(y_coeffs)
    total_len = len(X)
    train_idx, val_test_idx = train_test_split(
        np.array(range(total_len)),
        test_size=train_val_test[1]+train_val_test[2],
        random_state=42
    )
    val_idx, test_idx = train_test_split(
        val_test_idx,
        test_size=(train_val_test[2] / (train_val_test[1]+train_val_test[2]))
    )
    X_train = [X[i] for i in train_idx]
    y_taylor_train = [y_taylor[i] for i in train_idx]
    y_coeffs_train = [y_coeffs[i] for i in train_idx]

    X_val = [X[i] for i in val_idx]
    y_taylor_val = [y_taylor[i] for i in val_idx]
    y_coeffs_val = [y_coeffs[i] for i in val_idx]

    X_test = [X[i] for i in test_idx]
    y_taylor_test = [y_taylor[i] for i in test_idx]
    y_coeffs_test = [y_coeffs[i] for i in test_idx]

In [11]:
X_pickle_file_train = "data.nosync/data_train.pickle"
y_taylor_pickle_file_train = "data.nosync/data_taylor_train.pickle"
y_coeffs_pickle_file_train = "data.nosync/data_coeffs_train.pickle"

X_pickle_file_small_train = "data.nosync/data_train_small.pickle"
y_taylor_pickle_file_small_train = "data.nosync/data_taylor_train_small.pickle"
y_coeffs_pickle_file_small_train = "data.nosync/data_coeffs_train_small.pickle"

X_pickle_file_val = "data.nosync/data_val.pickle"
y_taylor_pickle_file_val = "data.nosync/data_taylor_val.pickle"
y_coeffs_pickle_file_val = "data.nosync/data_coeffs_val.pickle"

X_pickle_file_test = "data.nosync/data_test.pickle"
y_taylor_pickle_file_test = "data.nosync/data_taylor_test.pickle"
y_coeffs_pickle_file_test = "data.nosync/data_coeffs_test.pickle"

In [12]:
if export_data:
    with open(X_pickle_file_train, "wb") as f:
        pickle.dump(X_train, f)
    with open(y_taylor_pickle_file_train, "wb") as f:
        pickle.dump(y_taylor_train, f)
    with open(y_coeffs_pickle_file_train, "wb") as f:
        pickle.dump(y_coeffs_train, f)

    with open(X_pickle_file_small_train, "wb") as f:
        pickle.dump(X_train[0:100], f)
    with open(y_taylor_pickle_file_small_train, "wb") as f:
        pickle.dump(y_taylor_train[0:100], f)
    with open(y_coeffs_pickle_file_small_train, "wb") as f:
        pickle.dump(y_coeffs_train[0:100], f)

    with open(X_pickle_file_val, "wb") as f:
        pickle.dump(X_val, f)
    with open(y_taylor_pickle_file_val, "wb") as f:
        pickle.dump(y_taylor_val, f)
    with open(y_coeffs_pickle_file_val, "wb") as f:
        pickle.dump(y_coeffs_val, f)

    with open(X_pickle_file_test, "wb") as f:
        pickle.dump(X_test, f)
    with open(y_taylor_pickle_file_test, "wb") as f:
        pickle.dump(y_taylor_test, f)
    with open(y_coeffs_pickle_file_test, "wb") as f:
        pickle.dump(y_coeffs_test, f)

In [13]:
if (not load_data_from_scratch) & (not load_full_data):
    with open(X_pickle_file_small_train, "rb") as f:
        X_train = pickle.load(f)
    with open(y_taylor_pickle_file_small_train, "rb") as f:
        y_taylor_train = pickle.load(f)
    with open(y_coeffs_pickle_file_small_train, "rb") as f:
        y_coeffs_train = pickle.load(f)

    with open(X_pickle_file_test, "rb") as f:
        X_test = pickle.load(f)
    with open(y_taylor_pickle_file_test, "rb") as f:
        y_taylor_test = pickle.load(f)
    with open(y_coeffs_pickle_file_test, "rb") as f:
        y_coeffs_test = pickle.load(f)

elif load_full_data:
    with open(X_pickle_file_train, "rb") as f:
        X_train = pickle.load(f)
    with open(y_taylor_pickle_file_train, "rb") as f:
        y_taylor_train = pickle.load(f)
    with open(y_coeffs_pickle_file_train, "rb") as f:
        y_coeffs_train = pickle.load(f)
    with open(X_pickle_file_val, "rb") as f:
        X_val = pickle.load(f)
    with open(y_taylor_pickle_file_val, "rb") as f:
        y_taylor_val = pickle.load(f)
    with open(y_coeffs_pickle_file_val, "rb") as f:
        y_coeffs_val = pickle.load(f)
    with open(X_pickle_file_test, "rb") as f:
        X_test = pickle.load(f)
    with open(y_taylor_pickle_file_test, "rb") as f:
        y_taylor_test = pickle.load(f)
    with open(y_coeffs_pickle_file_test, "rb") as f:
        y_coeffs_test = pickle.load(f)

In [14]:
ic(len(X_train))
ic(len(X_val))

ic(len(X_test))

ic(len(y_taylor_train))
ic(len(y_taylor_val))
ic(len(y_taylor_test))

ic| len(X_train): 24000
ic| len(X_val): 3000
ic| len(X_test): 3000
ic| len(y_taylor_train): 24000
ic| len(y_taylor_val): 3000
ic| len(y_taylor_test): 3000


3000

In [15]:
X_prefix_train = [sympy_to_prefix(x) for x in X_train]
y_taylor_prefix_train = [sympy_to_prefix(y) for y in y_taylor_train]

X_prefix_val = [sympy_to_prefix(x) for x in X_val]
y_taylor_prefix_val = [sympy_to_prefix(y) for y in y_taylor_val]

X_prefix_test = [sympy_to_prefix(x) for x in X_test]
y_taylor_prefix_test = [sympy_to_prefix(y) for y in y_taylor_test]

In [53]:
if check_prefix_recovery:
    X_prefix_train_recovered = [prefix_to_sympy(x) for x in X_prefix_train]
    y_taylor_prefix_train_recovered = [prefix_to_sympy(y) for y in y_taylor_prefix_train]

    X_prefix_test_recovered = [prefix_to_sympy(x) for x in X_prefix_test]
    y_taylor_prefix_test_recovered = [prefix_to_sympy(y) for y in y_taylor_prefix_test]


    assert X_prefix_train_recovered == X_train
    assert y_taylor_prefix_train_recovered == y_taylor_train
    print("All train data correctly recovered with `prefix_to_sympy`")

    assert X_prefix_test_recovered == X_test
    assert y_taylor_prefix_test_recovered == y_taylor_test

    print("All test data correctly recovered with `prefix_to_sympy`")

In [54]:
def clean_data(X, y, max_len=200, del_manual_idxs=[]):
    """ Clean up data
    - manually remove indices from `del_manual_idxs`
    - remove where prefix notation is longer than 200
    """
    previous_length = len(X)

    X = [val for i, val in enumerate(X) if i not in del_manual_idxs]
    y = [val for i, val in enumerate(y) if i not in del_manual_idxs]

    X_too_long_idxs = too_long(X, max_len)
    X = [val for i, val in enumerate(X) if i not in X_too_long_idxs]
    y = [val for i, val in enumerate(y) if i not in X_too_long_idxs]

    y_too_long_idxs = too_long(y, max_len)
    X = [val for i, val in enumerate(X) if i not in y_too_long_idxs]
    y = [val for i, val in enumerate(y) if i not in y_too_long_idxs]

    total_removed = len(del_manual_idxs) + len(X_too_long_idxs) + len(y_too_long_idxs)
    del_percentage = total_removed / previous_length
    print("Deleted ", total_removed, "elements", "of", previous_length)
    print("That's ", round(del_percentage*100, 1), "%")

    return (X, y)


def too_long(data, max_len):
    """return indices where len(data)>max_len"""
    return [i for (i, x) in enumerate(data) if len(x)>max_len]

In [57]:
del_manual_idxs = [
]
max_len = 200
print("train:")
X_prefix_train, y_taylor_prefix_train = clean_data(X_prefix_train, y_taylor_prefix_train, del_manual_idxs=del_manual_idxs, max_len=max_len)

print("val:")
X_prefix_val, y_taylor_prefix_val = clean_data(X_prefix_val, y_taylor_prefix_val, del_manual_idxs=del_manual_idxs, max_len=max_len)

print("test:")
X_prefix_test, y_taylor_prefix_test = clean_data(X_prefix_test, y_taylor_prefix_test, max_len=max_len)

train:
Deleted  0 elements of 21621
That's  0.0 %
val:
Deleted  322 elements of 3000
That's  10.7 %
test:
Deleted  0 elements of 2705
That's  0.0 %


In [58]:
len(X_prefix_train)

21621

# Vectorization

In [59]:
def make_one_string(word_arr, start_end=False):
    ret = functools.reduce(lambda a, b: a+ " " + b, word_arr)
    if start_end:
        ret = "[start]" + " " + ret + " " + "[end]"
    return ret

In [60]:
from tensorflow.keras.layers import TextVectorization

In [61]:
# vocab_size = 1000
sequence_length = 200

numbers_tokens = ['int ', 's+', 's-'] +[str(i) for i in range(0,10)]
numbers_tokens = make_one_string(numbers_tokens)

In [62]:
X_vectorize_layer = TextVectorization(
    # max_tokens=vocab_size,
    standardize=None,
    split='whitespace',
    output_mode='int',
    output_sequence_length=sequence_length)

y_vectorize_layer = TextVectorization(
    # max_tokens=vocab_size,
    standardize=None,
    split='whitespace',
    output_mode='int',
    output_sequence_length=sequence_length+1)

2022-06-01 12:11:16.688967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-01 12:11:16.689144: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 12:11:16.689189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-01 12:11:16.689225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-01 12:11:16.689261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [63]:
X_prefix_train_str = [make_one_string(x) for x in X_prefix_train]
y_prefix_train_str = [make_one_string(y, start_end=True) for y in y_taylor_prefix_train]

X_prefix_val_str = [make_one_string(x) for x in X_prefix_val]
y_prefix_val_str = [make_one_string(y, start_end=True) for y in y_taylor_prefix_val]

X_prefix_test_str = [make_one_string(x) for x in X_prefix_test]
y_prefix_test_str = [make_one_string(y, start_end=True) for y in y_taylor_prefix_test]

In [64]:
X_prefix_train_str[0]

'mul x pow add d add sinh x tan x int s- 1'

In [65]:
y_prefix_train_str[0]

'[start] add mul x pow d int s- 1 add mul int s- 2 mul pow d int s- 2 pow x int s+ 2 add mul int s+ 4 mul pow d int s- 3 pow x int s+ 3 mul mul int s- 1 pow int s+ 2 4 int s- 1 mul pow d int s- 2 mul pow x int s+ 4 add int s+ 1 2 mul int s+ 1 9 2 pow d int s- 2 [end]'

In [66]:
numbers = [str(i) for i in range(0,10)]
numbers = 'int s+ s_ ' + make_one_string(numbers)
X_vectorize_layer.adapt([numbers]+X_prefix_train_str)
y_vectorize_layer.adapt([numbers]+y_prefix_train_str)

In [67]:
X_enc0 = X_vectorize_layer(X_prefix_train_str[0:1])
X_enc0

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[ 3,  2,  4,  8, 10,  8, 19,  2, 20,  2,  5,  9,  6,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]])>

In [68]:
y_enc0 = y_vectorize_layer(y_prefix_train_str[0:1])
y_enc0

<tf.Tensor: shape=(1, 201), dtype=int64, numpy=
array([[11,  8,  3, 10,  5, 16,  2,  7,  6,  8,  3,  2,  7,  9,  3,  5,
        16,  2,  7,  9,  5, 10,  2,  4,  9,  8,  3,  2,  4, 13,  3,  5,
        16,  2,  7, 14,  5, 10,  2,  4, 14,  3,  3,  2,  7,  6,  5,  2,
         4,  9, 13,  2,  7,  6,  3,  5, 16,  2,  7,  9,  3,  5, 10,  2,
         4, 13,  8,  2,  4,  6,  9,  3,  2,  4,  6, 32,  9,  5, 16,  2,
         7,  9, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0]])>

In [69]:
X_vocabulary = np.array(X_vectorize_layer.get_vocabulary())
y_vocabulary = np.array(y_vectorize_layer.get_vocabulary())

In [70]:
def unvectorize(sentence, vocab):
    return list(vocab[tuple(sentence)])

In [71]:
X_train_vec = X_vectorize_layer(X_prefix_train_str)
y_train_vec = y_vectorize_layer(y_prefix_train_str)

X_val_vec = X_vectorize_layer(X_prefix_val_str)
y_val_vec = y_vectorize_layer(y_prefix_val_str)

X_test_vec = X_vectorize_layer(X_prefix_test_str)
y_test_vec = y_vectorize_layer(y_prefix_test_str)

In [72]:
batch_size=128

def format_dataset(X, y):
    return ({"encoder_inputs": X, "decoder_inputs": y[:, :-1],}, y[:, 1:])

def make_dataset(X, y):
    dataset = format_dataset(X,y)
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    dataset = dataset.batch(batch_size)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(X_train_vec, y_train_vec)
val_ds = make_dataset(X_val_vec, y_val_vec)

In [73]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 200)
inputs["decoder_inputs"].shape: (128, 200)
targets.shape: (128, 200)


2022-06-01 12:11:59.620685: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Transformer

In [74]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [35]:
pip install keras_tuner

Note: you may need to restart the kernel to use updated packages.


In [75]:
import keras_tuner as kt

In [76]:
def model_builder(hp):
    # embed_dim = 256
    # latent_dim = 512*2
    # num_heads = 7
    vocab_size = 200

    embed_dim = hp.Int('embed_dim', min_value=256, max_value=512, step=56)
    num_heads = hp.Int('num_heads', min_value=4, max_value=10, step=1)
    latent_dim = hp.Int('latent_dim', min_value=256, max_value=512*4, step=256)

    
    encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
    encoder = tf.keras.Model(encoder_inputs, encoder_outputs)
    
    decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
    encoded_seq_inputs = tf.keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
    x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
    x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
    decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
    
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = tf.keras.Model(
        [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
    )
    transformer.compile(
        "rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return transformer

In [57]:
tuner = kt.Hyperband(model_builder,
                    overwrite=True,
                    objective="val_loss",
                    max_epochs=200,
                    factor=3,
                    directory="models/2022-05-31-TaylorModelPrefix",
                    project_name="2022-05-31-TaylorModelPrefix")

In [79]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=50)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=1e-5) 

tuner.search(train_ds,
             validation_data=val_ds,
             epochs=200,
             callbacks=[
                 stop_early,
                # tf.keras.callbacks.TensorBoard("logs/2022-05-31-TaylorModelPrefix")
             ],
             )


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
312               |?                 |embed_dim
10                |?                 |num_heads
2048              |?                 |latent_dim
3                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
4                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/3
 8/62 [==>...........................] - ETA: 6:39 - loss: 1.7538 - accuracy: 0.1172

KeyboardInterrupt: 

In [40]:
tmp_hps = kt.HyperParameters()
tmp_hps.values = {
    'embed_dim': 200,
    'num_heads': 5,
    'latent_dim': 512,
           }

In [41]:
model = model_builder(tmp_hps)
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_2 (Positi  (None, None, 200)   80000       ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_1 (Transfo  (None, None, 200)   1009512     ['positional_embedding_

In [ ]:
tf.keras.callbacks.ModelCheckpoint(
    "models/2022-05-31-TaylorModelPrefix_retrained_best/",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
)

In [43]:
model.fit(train_ds, epochs=2, validation_data=val_ds)

Epoch 1/2
62/62 [==============================] - 122s 2s/step - loss: 0.6221 - accuracy: 0.4277 - val_loss: 0.4751 - val_accuracy: 0.5049
Epoch 2/2
62/62 [==============================] - 114s 2s/step - loss: 0.4131 - accuracy: 0.5193 - val_loss: 0.4325 - val_accuracy: 0.5341


In [44]:
model.save_weights("models/test_model_weights.h5")

In [45]:
model_reloaded = model_builder(tmp_hps)
model_reloaded.load_weights("models/test_model_weights.h5")

# Testing

In [120]:
max_decoded_sentence_length = 200
def apply_transformer(X, y_vectorize_layer):
    decoded_sentence = ["[start]"]
    vocab = y_vectorize_layer.get_vocabulary()
    X_tensor = tf.convert_to_tensor([X])

    # for i in range(max_decoded_sentence_length):
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = y_vectorize_layer(decoded_sentence)[:,0:max_decoded_sentence_length]
        y_pred = model_reloaded([X_tensor, tokenized_target_sentence])[0,i,:]
        y_index = np.argmax(y_pred)
        y = vocab[y_index]
        decoded_sentence[0] = decoded_sentence[0] + " " + y

        if y == "[end]":
            break
    return decoded_sentence[0][1:-1] # no start/end
    # return  decoded_sentence

In [121]:
def token_accuracy(y_true, y_pred, verbose=0):
    """
    compare two arrays and check how many entries are the same at the same position
    """
    max_ind = np.min([len(y_true), len(y_pred)])
    correct_ctr = 0
    # ignore [start] and [end]
    max_correct = len(y_pred) - 2
    for i in range(1, max_ind-1):
        if verbose: ic([y_true[i], y_pred[i]])
        if y_true[i] == y_pred[i]:
            correct_ctr += 1
    return correct_ctr / max_correct

In [122]:
y_val_pred = [apply_transformer(x, y_vectorize_layer).split() for x in X_val_vec[0:10]]

In [127]:
token_acc_list = [token_accuracy(y_true, y_pred) for y_true, y_pred in zip(y_taylor_prefix_val[0:10], y_test_pred[0:10])]
token_acc = np.mean(token_acc_list)
ic(token_acc_list)
ic(token_acc)

ic| token_acc_list: [0.2,
                     0.03314917127071823,
                     0.07103825136612021,
                     0.06097560975609756,
                     0.0582010582010582,
                     0.06077348066298342,
                     0.11538461538461539,
                     0.05025125628140704,
                     0.0,
                     0.09090909090909091]
ic| token_acc: 0.07406825338320909


0.07406825338320909